### Get the Personalize boto3 Client

All of the code that we are using in this lab is Python, but any langugage supported by SageMaker could be used.  In this initial piece of code we are loading in the library dependencies that we need for the rest of the lab:

- **boto3** - standard Python SDK that wraps the AWS CLI
- **json** - used to manipulate JSON structures used by our API calls
- **numpy** and **pandas** - standard libraries used by Data Scientists
- **time** - used for some time manipulation calls
- **display** - to display images in the notebook

In [25]:
import boto3
import json
import numpy as np
import pandas as pd
import time
from IPython.display import Image

personalize = boto3.client(service_name='personalize')
personalize_runtime = boto3.client(service_name='personalize-runtime')

# If you have to rerun the notebook, increment this suffix to avoid conflicts
suffix = '1'

# Retrieve suffix used by background notebook
%store -r bg_suffix

### Specify a Bucket and Data Output Location

For this demo, we'll choose to upload data to Amazon Personalize directly from an S3 bucket.  Hence, you need to create a new bucket - please name your bucket before running this Code cell, overriding what is shown in the code cell, and you need to ensure that the bucket name is globally unique; for this lab we recommend using your name or initials, followed by *-bp-personalize-lab-*, as that is likely to be unique.

If the bucket already exists - such as if you execute this code cell a second time - then it will not create a new bucket, and will not make any changes to the exsting bucket.

In [2]:
# Replace with the name of your S3 bucket
bucket = "{your-prefix}-bp-personalize-lab-" + suffix

s3 = boto3.client('s3')
if boto3.resource('s3').Bucket(bucket).creation_date is None:
    s3.create_bucket(ACL = "private", Bucket = bucket)
    print("Creating bucket: {}".format(bucket))

Creating bucket: cjv-bp-personalize-lab-1


### Download, Prepare, and Upload Training Data

#### Download and Explore the Dataset

In this step we download the gas station data set zip-file and extract it - it will go in the same location as the physical notebook *.ipynb* file.  We use the **pandas** library to read in the *gas_station_data_5k.csv* file, which contains all of the in-store beverage purchases; this file consists of a user ID (vehicle category), item ID (beverage), a timestamp.  The dataset has been trimmed to 5k entries.

In [3]:
filename = 'gas_station_data_5k'

!wget -N https://bp-personalize-lab-2020.s3.amazonaws.com/gas_station_data_5k.zip
!unzip -o gas_station_data_5k.zip

data = pd.read_csv('./gas_station_data_5k.csv', sep=',', names=['USER_ID', 'ITEM_ID', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

--2020-03-16 16:30:23--  https://bp-personalize-lab-2020.s3.amazonaws.com/gas_station_data_5k.zip
Resolving bp-personalize-lab-2020.s3.amazonaws.com (bp-personalize-lab-2020.s3.amazonaws.com)... 52.216.107.180
Connecting to bp-personalize-lab-2020.s3.amazonaws.com (bp-personalize-lab-2020.s3.amazonaws.com)|52.216.107.180|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘gas_station_data_5k.zip’ not modified on server. Omitting download.

Archive:  gas_station_data_5k.zip
  inflating: gas_station_data_5k.csv  


,USER_ID,ITEM_ID,TIMESTAMP
0,10,8,874725485
1,2,21,874728370
...,...,...,...
4998,12,6,879440475
4999,12,7,879440545


##### Prepare and Upload Data

We don't actually need all of the purchase data.  We would like to recommend non-alcoholic beverages, as we do not know the age of the customer or the time of day.  We would not want to promote those beverages to minors or on a Sunday.  Hence, we're going to use **pandas** to drop the entries related to alcoholic beverages - we're left with a subset of the original, which are safe to promote.

Additionally, the purchases are quite old - they are from August 1997 to April 1998.  Some of the Amazon Personalize recipies react differently depending upon the age of the interactions - for instance, the _Similar Items_ recipe has several hyperparameters around how to handle 'decaying' interactions.  In order to make this lab easier, and not have to worry about these hyperparameters, we are shifting all of the purchase timestamps to be from August 2018 up until April 2019.

We then write that out to a file named and upload it into our S3 bucket.  

This is the minimum amount of data that Amazon Personalize needs to train a model on - you need just 1500 rows of user/item/timestamp interactions, but we still have many thousands of entries left from our original dataset.  This file is known in Amazon Personalize as an **Interactions** data file.  Other data files could be usable, such as ones that define additional metadata about the beverages and another that defines demographic data about the user (such as age, gender and location).  In this lab we do not have this available.

In [4]:
filename = "gas_station_data.csv"

data = data[data['ITEM_ID'] < 15]                  # keep only non-alcoholic beverages
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']]   # select columns that match the columns in the schema below
data['TIMESTAMP'] = data['TIMESTAMP'] + 660833618  # make purchases end 1st April 2019 rather than 23rd April 1998
data.to_csv(filename, index=False)

# Upload data to S3
boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

### Create Schema

Amazon Personalize uses *Schemas* to tell it how to interpret your data files.  This step defines the schema for our Interations file, which consists solely of a `USER_ID`, `ITEM_ID` and `TIMESTAMP`.  Once defined we pass it into Personalize for use.

In [6]:
schema_name = "bp-pers-schema-" + suffix

schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

# Create the Schema in Personalize
create_schema_response = personalize.create_schema(
    name = schema_name,
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:471551772664:schema/bp-pers-schema-1",
  "ResponseMetadata": {
    "RequestId": "f0cb9a53-7626-496a-a2f9-c86ec5d3088b",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 16 Mar 2020 16:31:08 GMT",
      "x-amzn-requestid": "f0cb9a53-7626-496a-a2f9-c86ec5d3088b",
      "content-length": "82",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Create and Wait for Dataset Group

Now that we have defined a schema, and we have our Interactions data file, we can import the data into Personalize.  But first we have to define a *Dataset Group*, which is essentially a collection of imported data files, trained models and campaigns - each Dataset Group can contain one, and only one, Interaction, Item Metadata and User Demographic file.  When you train a model Personalize will use **all** data files present within its Dataset Group.

#### Create Dataset Group

In [7]:
dataset_name = "bp-personalize-dataset-group" + suffix

create_dataset_group_response = personalize.create_dataset_group(
    name = dataset_name
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:471551772664:dataset-group/bp-pers-dataset-group1",
  "ResponseMetadata": {
    "RequestId": "99d5872b-63a9-4e39-8dbb-f6d478984a15",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 16 Mar 2020 16:31:20 GMT",
      "x-amzn-requestid": "99d5872b-63a9-4e39-8dbb-f6d478984a15",
      "content-length": "101",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Dataset Group to Have ACTIVE Status

A number of Personalize API calls do take time, hence the calls are asynchronous.  Before we can continue with the next stage we need to poll the status of the `create_dataset_group()` call from the previous code cell - once the Dataset Group is active then we can continue.  **NOTE: this step should not take more than 1-2 minutes to complete**

In [8]:
status = None
max_time = time.time() + 3*60*60 # wait up to 3 hours

while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


### Create Dataset

We now have to create our dataset for the Interactions file.  This step does not actually import any data, rather it creates an internal structure for the data to be imported into.

In [9]:
dataset_type = "INTERACTIONS"

create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn,
    name="bp-pers-dataset-" + suffix
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:471551772664:dataset/bp-pers-dataset-group1/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "02303a4a-4e53-4a82-aa8f-76cf0fe64c06",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 16 Mar 2020 16:36:00 GMT",
      "x-amzn-requestid": "02303a4a-4e53-4a82-aa8f-76cf0fe64c06",
      "content-length": "103",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Prepare, Create, and Wait for Dataset Import Job

#### Attach policy to S3 bucket

Whilst we have created an S3 bucket, and our Interactions data file is sat there waiting to be imported, we have a problem - you may have full access to the bucket via the AWS console or APIs, but the Amazon Personalize service does not.  Hence, you have to create an S3 bucket policy that explicitly grants the service access to the `GetObject` and `ListBucket` commands in S3.  This code step creates such a policy and attaches it to your S3 bucket.

Note, any Personalize API calls that need to access you S3 bucket need to be done using an IAM role that gives it permission - this step simply allows the service to access the bucket if, and only if, roles with appropriate permissions are used.

In [10]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

#### Find Personalize S3 Role ARN

As part of the AWS Event Engine process we have defined an IAM role that gives Personalize the ability to access S3 buckets - as mentioned this is needed as well as the S3 bucket policy.  As the Event Engine creates the IAM role via CloudFormation it will always have an essentially random numeric suffix, so we cannot hard-code it into the lab.  This code cell looks for any any service role that has the name _PersonalizeRoleForLab_ in it and selects it as the ARN that we need.

In [11]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleForLab"
role_list = iam.list_roles()

for role in role_list['Roles']:
    if role_name in (role['Arn']):
        role_arn = (role['Arn'])
        
role_arn

'arn:aws:iam::471551772664:role/pers-iam-role-PersonalizeRoleForLab-CO81EV7IVHGP'

#### Create Dataset Import Job

This pulls together the information that we have on our Dataset, on our S3 bucket, on our Interactions file and a suitable role for Personalize, and then triggers the actual data import process.

In [12]:
dataSource = {"dataLocation": "s3://{}/{}".format(bucket, filename)}

create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "bp-personalize-import-job" + suffix,
    datasetArn = dataset_arn,
    dataSource = {"dataLocation": "s3://{}/{}".format(bucket, filename)},
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:471551772664:dataset-import-job/bp-personalize-import-job1",
  "ResponseMetadata": {
    "RequestId": "2813d85e-382d-4468-b633-8b8103365e8a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 16 Mar 2020 16:36:14 GMT",
      "x-amzn-requestid": "2813d85e-382d-4468-b633-8b8103365e8a",
      "content-length": "114",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Dataset Import Job and Dataset Import Job Run to Have ACTIVE Status

We now poll the status of Interactions file import job, as until it is complete we cannot continue.  **Note: this can take anything between 12-25 minutes to complete.** We will only wait for 3 minutes, then use the same dataset imported in the background, which should have completed by now.

In [13]:
status = None

max_time = time.time() + 3*60 # 3 minutes

while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS


### Select Recipe

There are many different algorithm recipes available within Personalize, this is a list of all supported algorithms at the time of the workshop.  We are going to select the standard HRNN recipe, which only needs the Interactions file and not the Item metadata or User demographic files.

In [14]:
recipe_list = [
    "arn:aws:personalize:::recipe/aws-hrnn",
    "arn:aws:personalize:::recipe/aws-hrnn-coldstart",
    "arn:aws:personalize:::recipe/aws-hrnn-metadata",
    "arn:aws:personalize:::recipe/aws-personalized-ranking",
    "arn:aws:personalize:::recipe/aws-popularity-count",
    "arn:aws:personalize:::recipe/aws-sims"
]

recipe_arn = recipe_list[0]
print(recipe_arn)

arn:aws:personalize:::recipe/aws-hrnn


### Create and Wait for Solution

With our data imported we can now train our ML solution.  This consists of just a single API to Personalize, where we specify the Dataset to use.

#### Create Solution

In [15]:
%store -r dataset_group_arn

create_solution_response = personalize.create_solution(
    name = "bp-beverage-solution" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:471551772664:solution/bp-beverage-solution1",
  "ResponseMetadata": {
    "RequestId": "9d277a2f-c771-42be-8f01-b6d92489d99b",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 16 Mar 2020 16:41:47 GMT",
      "x-amzn-requestid": "9d277a2f-c771-42be-8f01-b6d92489d99b",
      "content-length": "91",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create Solution Version

In [16]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:471551772664:solution/bp-beverage-solution1/ec3dda5f",
  "ResponseMetadata": {
    "RequestId": "a229c22f-7f97-496e-8128-21563929d613",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 16 Mar 2020 16:41:52 GMT",
      "x-amzn-requestid": "a229c22f-7f97-496e-8128-21563929d613",
      "content-length": "107",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Solution to Have ACTIVE Status

We now poll the status of solution creation job, as until it is complete we cannot continue. **Note: this can take anything between 25-50 minutes to complete**. Again, we will wait for a few minutes only, then use the solution generated in the background.

In [17]:
status = None
#max_time = time.time() + 3*60*60 # 3 hours
max_time = time.time() + 3*60 # 3 minutes

while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS


#### Create Additional Solutions in the Console

Whilst you're waiting for this to complete, jump back into the original Lab Guidebook - there we will walk you through creating two additional solutions in parallel using the same dataset; one for Personalized Rankings and one for Item-to-Item Similarities (or SIMS), both of which can be used in the final application.  Once you've begun to create both additional solutions you can come back here and continue.

#### Get Metrics of Solution

Once the solution is built you can look up the various metrics that Personalize provides - this allows you to see how well a model has been trained.  If you are re-training models after the acquisition of new data then these metrics can tell you if the models are training equally as well as before, better than before or worse than before, giving you the information that you need in order to decide whether or not to push a new model into Production.  You can also compare results across multiple different algorithm recipes, helping you choose the best performing one for you particular dataset.

You can find details on each of the metrics in our [documentation](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html).

In [18]:
%store -r solution_version_arn

get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:471551772664:solution/bp-beverage-solutionbackground1/b5bb675f",
  "metrics": {
    "coverage": 1.0,
    "mean_reciprocal_rank_at_25": 0.9,
    "normalized_discounted_cumulative_gain_at_10": 0.7594,
    "normalized_discounted_cumulative_gain_at_25": 0.8657,
    "normalized_discounted_cumulative_gain_at_5": 0.7084,
    "precision_at_10": 0.62,
    "precision_at_25": 0.336,
    "precision_at_5": 0.68
  },
  "ResponseMetadata": {
    "RequestId": "740daa50-c420-4bb8-acaa-fe81779d6c77",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 16 Mar 2020 19:19:05 GMT",
      "x-amzn-requestid": "740daa50-c420-4bb8-acaa-fe81779d6c77",
      "content-length": "404",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Create and Wait for Campaign

A trained model is exactly that - just a model.  In order to use it you need to create an API endpoint, and you do this by creating a *Campaign*.  A Campaign simply provides the endpoint for a specific version of your model, and as such you are able to host endpoint for multiple versions of your models simultaneously, allowing you to do things like A/B testing of new models.

At the campaign level we specify the the minimum deployed size of the inference engine in terms of transactions per second - whilst this engine can scale up and down dynamically it will never scale below this level, but please note that pricing for Personalize is heavily based around the number of TPS currently deployed.

#### Create campaign

In [19]:
create_campaign_response = personalize.create_campaign(
    name = "bp-beverage-campaign" + suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:471551772664:campaign/bp-beverage-campaign1",
  "ResponseMetadata": {
    "RequestId": "e58fe2b1-b152-4a53-879e-c23fff6da9fd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 16 Mar 2020 19:19:08 GMT",
      "x-amzn-requestid": "e58fe2b1-b152-4a53-879e-c23fff6da9fd",
      "content-length": "91",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Campaign to Have ACTIVE Status

We now poll the status of Campaign creation job, as until it is complete we cannot continue.  **Note: this can take anything between 3-15 minutes to complete**

In [20]:
status = None
max_time = time.time() + 3*60*60 # 3 hours

while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


### Get Recommendations

Finally, we have a deployed Campaign end-point, which hosts a specific version of our trained model - we are now able to make recommendation inference requests against it.  However, the Personalize recommendation calls just return itemID values - it returns no context around the title of the movie,  Hence, we use our pre-loaded version of the *beverages.csv* file that contains the beverage types - we load in the file via **pandas** library calls and pick out a random drink ID (ITEM_ID) and vehicle ID (USER_ID) from our training data.  This info is displayed, along with the name of the beverage.

#### Select a User and an Item

In [121]:
users = pd.read_csv('./vehicles.csv', sep=',', usecols=[0,1], header=None)
users.columns = ['USER_ID', 'TITLE']

items = pd.read_csv('./beverages.csv', sep=',', usecols=[0,1], header=None)
items.columns = ['ITEM_ID', 'TITLE']

user_id, item_id, _ = data.sample().values[0]

user_title = users.loc[users['USER_ID'] == item_id].values[0][-1]
item_title = items.loc[items['ITEM_ID'] == item_id].values[0][-1]

print("User:               {}".format(user_title))
print("Previous Purchase:  {}".format(item_title))

User:               SUV, New
Previous Purchase:  Sports Drink


#### Call GetRecommendations

The last thing we do is actually make a Personalize recommendations inference call - as you can see from the Code cell this is literally a single line of code with a userID and itemID as input variables (although, strictly speaking, you only need the userID for the datasets that we have).

The inference call returns a list of up to 25 itemIDs from the training set - we take that and look up the corresponding beverage name from the *beverages.csv* file and display them; this is far more useful than just a list of ID values.

In [122]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id)
)

item_list = get_recommendations_response['itemList']
title_list = [items.loc[items['ITEM_ID'] == np.int(item['itemId'])].values[0][-1] for item in item_list]

print("Recommendations for {}: {}".format(user_title, json.dumps(title_list, indent=2)))

%store -r images

print("\nSuggestion #1:")
display(Image(url=images[title_list[0]]["url"], width=150, height=150))
print("\nSuggestion #2:")
display(Image(url=images[title_list[1]]["url"], width=150, height=150))
print("\nSuggestion #3:")
display(Image(url=images[title_list[2]]["url"], width=150, height=150))


Recommendations for SUV, New: [
  "Coffee, Bottled",
  "Coca-Cola/Pepsi",
  "DrPepper/MrPibb",
  "Energy Drink",
  "Mountain Dew/Sierra Mist",
  "Sports Drink",
  "Fanta",
  "Tea",
  "Sprite/7Up",
  "Juice",
  "Root Beer",
  "Coffee, Store",
  "Water, Bottled",
  "Milk"
]

Suggestion #1:



Suggestion #2:



Suggestion #3:
